In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate
import Utils
from Utils import Constants
import cv2
from facenet_pytorch import InceptionResnetV1
from Models import *
from DataLoaders import *

In [4]:
train_labels = pd.read_csv('train_data_augmented_balanceddual.csv')
validation_labels = pd.read_csv('validation_data_augmented_balanceddual.csv')
train_labels.shape, validation_labels.shape

((6842, 166), (1711, 166))

In [5]:
def get_model(file,model=None):
    if model is None:
        model = torch.load(Constants.model_folder + file).to(torch.device('cpu'))
    model.load_state_dict(torch.load(Constants.model_folder + file + '_states'))
    model.eval()
    return model
get_model('abstractmodeltriplet_decoder__st600_a400_g400_std2_ad2_gd2_balanced')

TripletModel(
  (encoder): TripletFacenetEncoder(
    (base_model): InceptionResnetV1(
      (conv2d_1a): BasicConv2d(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (conv2d_2a): BasicConv2d(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (conv2d_2b): BasicConv2d(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv2d_3b): BasicConv2d(
        (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), 

In [6]:
def save_train_history(model,history,root=''):
    model_name = model.get_identifier()
    
    df = pd.DataFrame(history)
    df['model'] = model_name
    string = root + 'results/history_' + model_name + '3.csv'
    df.to_csv(string,index=False)
    print('saved history to',string)
    return df, string

def train_model(model,
                train_df,
                validation_df,
                root,
                epochs=300,
                lr=.001,
                batch_size=200,
                patience = 20,
                loss_weights = [2,1,.5],
                save_path=None,
                histogram =False,
                upsample=False,
                random_upsample=True,
                softmax_upsample_weights=False,
                upsample_validation=False,
                random_upsample_validation=False,
                embedding_loss_weight = 1,
                classification_loss_weight = 1,
                starting_loss=100000,
                **kwargs,
               ):
    pretraining = classification_loss_weight <= .00001
    if save_path is None:
        save_path = root + 'models/'+ model.get_identifier()
        if random_upsample:
            
            save_path += '_rbalanced'
            if softmax_upsample_weights:
                save_path += 'soft'
        elif upsample:
            save_path += '_balanced'
        if pretraining:
            save_path += '_pretrain'
    if upsample:
        patience = int(patience/5) + 1
    train_loader = TripletFaceGenerator(train_df,Constants.data_root,
                                 batch_size=batch_size,
                                 upsample=upsample,
                                 random_upsample=random_upsample,
                                 softmax=softmax_upsample_weights,
                                 **kwargs)
    validation_loader = TripletFaceGenerator(validation_df,Constants.data_root,
                                      validation=True,
                                      batch_size=batch_size,
                                      upsample=upsample_validation,
                                     random_upsample=random_upsample_validation,
                                     softmax=softmax_upsample_weights,
                                      **kwargs)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train(True)
    
    loss_fn = torch.nn.CrossEntropyLoss()
    triplet_loss = torch.nn.TripletMarginLoss()
#     embedding_optimizer = torch.optim.Adam(model.encoder.parameters(), lr=lr)
#     decoder_optimizer = torch.optim.Adam(model.decoder.parameters(), lr=lr)
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    format_y = lambda y: y.long().to(device)
    
    def format_batch(inputs,grad=True):
        xb = []
        for xin in inputs:
            xin = xin.to(device)
            xin.requires_grad_(grad)
            xb.append(xin)
        return xb
    
    def embedding_step(m,xbatch): 
        base = m.encoder(xbatch[0])
        positive = m.encoder(xbatch[1])
        negative = m.encoder(xbatch[2])
        loss = triplet_loss(base,positive,negative)
        loss = torch.mul(loss,embedding_loss_weight)
        return base,loss
    
    
    def classifier_step(m,embedding,ytrue):
        outputs = m.decoder(embedding)
        losses = [loss_fn(ypred.float(),format_y(y)) for y,ypred in zip(ytrue,outputs)]
        l1 = torch.mul(loss_weights[0],losses[0])
        l2 =  torch.mul(loss_weights[1],losses[1])
        l3 =  torch.mul(loss_weights[2],losses[2])
        total_losses = l1 + l2 + l3
        total_loss = torch.mul(total_losses,classification_loss_weight)
        return outputs,total_losses
        
    def train_epoch(model):
        running_loss = 0
        running_embed_loss = 0
        running_accuracy = [0,0,0]
        curr_loss = 0
        count = 0
        for i, [x_batch, y_batch] in enumerate(train_loader):
            x_batch = format_batch(x_batch)
            optimizer.zero_grad()
            embedding,embedding_loss = embedding_step(model, x_batch)
            if pretraining:
                classification_loss = embedding_loss - embedding_loss
            else:
                outputs, classification_loss = classifier_step(model,embedding,y_batch)
            total_loss = classification_loss + embedding_loss
            total_loss.backward()
            optimizer.step()
            running_loss += classification_loss.item()
            running_embed_loss += embedding_loss.item()
            print('curr loss class',classification_loss.item(),'embed', embedding_loss.item(), 'step',i,' | ',end='\r')
            count += 1
            if not pretraining:
                with torch.no_grad():
                    for i,(y,ypred) in enumerate(zip(y_batch,outputs)):
                        accuracy = Utils.categorical_accuracy(ypred.float(),format_y(y))
                        running_accuracy[i] += accuracy.item()
        return running_loss/count,running_embed_loss/count, [a/count for a in running_accuracy]
    
    def val_epoch(model):
        running_loss = 0
        running_embed_loss = 0
        running_accuracy = [0,0,0]
        running_f1 = [0,0,0]
        count = 0
        with torch.no_grad():
            for i, [x_batch, y_batch] in enumerate(validation_loader):
                x_batch = format_batch(x_batch,grad=False)
                embedding,embedding_loss = embedding_step(model, x_batch)
                outputs, classification_loss = classifier_step(model,embedding,y_batch)
                running_loss += classification_loss.item()
                running_embed_loss += embedding_loss.item()
                count += 1
                for i,(y,ypred) in enumerate(zip(y_batch, outputs)):
                    accuracy = Utils.categorical_accuracy(ypred.float(),format_y(y))
                    f1, precision, recall = Utils.macro_f1(torch.argmax(ypred.float(),axis=1),format_y(y))
                    running_accuracy[i] += accuracy.item()
                    running_f1[i] += f1.item()
        return running_loss/count,running_embed_loss/count, [a/count for a in running_accuracy], [f/count for f in running_f1]
    shorten = lambda array: [np.round(a, 3) for a in array]
    
    best_val_loss = starting_loss
    steps_since_improvement = 0
    hist = []
    best_weights = model.state_dict()
    print('model being saved to',save_path)
    for epoch in range(epochs):
        print('epoch',epoch)
        model.train()
        avg_loss,avg_embed_loss, avg_acc = train_epoch(model)
        print('train loss', avg_loss,'train embed loss',avg_embed_loss, 'train accuracy', shorten(avg_acc))
        model.eval()
        val_loss,val_embed_loss, val_acc, val_f1 = val_epoch(model)
        print('val loss', val_loss, 'val_embed_loss', val_embed_loss, 
              'val accuracy', shorten(val_acc), 'val f1', shorten(val_f1))
        #don't save immediately in case I cancel training
        if best_val_loss > val_loss and epoch > 1:
            torch.save(model,save_path)
            torch.save(model.state_dict(),save_path+'_states')
            print('saving model')
            print(best_weights['encoder.base_model.conv2d_1a.conv.weight'].mean(), model.state_dict()['encoder.base_model.conv2d_1a.conv.weight'].mean())
            best_weights = model.state_dict()
            best_val_loss = val_loss
            steps_since_improvement = 0
        else:
            steps_since_improvement += 1
        
        hist_entry = {
            'epoch': epoch,
            'train_loss': avg_loss,
            'train_acc':avg_acc,
            'val_loss':val_loss,
            'val_acc': val_acc,
            'lr': lr,
            'loss_weights': '_'.join([str(l) for l in loss_weights])
        }
        hist.append(hist_entry)
        save_train_history(model,hist,root=root)
        if steps_since_improvement > patience:
            break
    model.load_state_dict(best_weights)
    return model,hist

def InceptionResnet(pretrained=None,**kwargs):
    m = InceptionResnetV1(pretrained=pretrained,classify=True,num_classes=10,**kwargs)
    m.get_identifier = lambda : 'inception' + (pretrained if pretrained is not None else '')
    return m
    
    
model = TripletModel(
    encoder=TripletFacenetEncoder(base_model=ResNet18()),
)

m,h = train_model(
#     TripletModel(),
#     get_model('abstractmodeltriplet_decoder__st600_a400_g400_std2_ad2_gd2_balanced'),
    model,
    train_labels,
    validation_labels,
    Constants.data_root,
    batch_size=50,
    loss_weights=[5,2,1],
    embedding_loss_weight=1,
    classification_loss_weight=0,
    lr=.001,
)
del m
h

(4869, 167)
(1209, 167)
model being saved to ../../data/models/dualencoder_resnet18_h400_ed3triplet_decoder__st600_a400_g400_std2_ad2_gd2_rbalanced_pretrain
epoch 0


/opt/conda/envs/rapids/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train loss 0.0 train embed loss 2.664711440096096 train accuracy [0.0, 0.0, 0.0]
val loss 15.125073738098145 val_embed_loss 29.042501792907714 val accuracy [0.022, 0.174, 0.591] val f1 [0.019, 0.093, 0.268]
saved history to ../../data/results/history_dualencoder_resnet18_h400_ed3triplet_decoder__st600_a400_g400_std2_ad2_gd23.csv
epoch 1
train loss 0.0 train embed loss 1.569779401531025 train accuracy [0.0, 0.0, 0.0]
val loss 14.991635131835938 val_embed_loss 1.257501859664917 val accuracy [0.023, 0.246, 0.593] val f1 [0.024, 0.106, 0.231]
saved history to ../../data/results/history_dualencoder_resnet18_h400_ed3triplet_decoder__st600_a400_g400_std2_ad2_gd23.csv
epoch 2
train loss 0.0 train embed loss 1.2533772423559306 train accuracy [0.0, 0.0, 0.0]
val loss 14.96999610900879 val_embed_loss 1.303478388786316 val accuracy [0.152, 0.214, 0.596] val f1 [0.077, 0.11, 0.229]


AttributeError: Can't pickle local object 'ResNet18.<locals>.<lambda>'

In [ ]:


DenseNet()

In [ ]:
tm.densenet161()